From [Suanlab](https://www.youtube.com/channel/UCFfALXX0DOx7zv6VeR5U_Bg)
# 한국환경공단 API 활용하기

## 한국환경공단 측정소정보
https://www.data.go.kr/iim/api/selectAPIAcountView.do

encoding: KJRTsEOIytUcYU%2BqXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx%2BrBGrAPjuxqrqHEJPbqdKeAlp8kNA%3D%3D

decoding:
KJRTsEOIytUcYU+qXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx+rBGrAPjuxqrqHEJPbqdKeAlp8kNA==



In [1]:
# # Python 샘플 코드 #


# from urllib2 import Request, urlopen
# from urllib import urlencode, quote_plus

# url = 'http://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList'
# queryParams = '?' + urlencode({ quote_plus('ServiceKey') : '서비스키', quote_plus('serviceKey') : '인증키(URL Encode)', quote_plus('returnType') : 'xml', quote_plus('numOfRows') : '100', quote_plus('pageNo') : '1', quote_plus('addr') : '서울', quote_plus('stationName') : '종로구' })

# request = Request(url + queryParams)
# request.get_method = lambda: 'GET'
# response_body = urlopen(request).read()
# print response_body

In [2]:

from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from xml.etree.ElementTree import parse
import xml.etree as ET
import pandas as pd
import xmltodict
import json

enc = 'KJRTsEOIytUcYU%2BqXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx%2BrBGrAPjuxqrqHEJPbqdKeAlp8kNA%3D%3D'
dec = 'KJRTsEOIytUcYU+qXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx+rBGrAPjuxqrqHEJPbqdKeAlp8kNA=='

url = 'http://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList'

params = '?' + urlencode({  quote_plus('serviceKey') : dec, 
                            quote_plus('returnType') : 'xml', 
                            quote_plus('numOfRows') : '100', 
                            quote_plus('pageNo') : '1', 
                            quote_plus('addr') : '서울'})



request = Request(url + params)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8') #DECODE 옵션넣어줘야 보임  
print(response_body)

<?xml version="1.0" encoding="UTF-8"?>
<response>
  <header>
    <resultCode>00</resultCode>
    <resultMsg>NORMAL_CODE</resultMsg>
  </header>
  <body>
    <items>
      <item>
        <dmX>37.5175623</dmX>
        <item>SO2, CO, O3, NO2, PM10, PM2.5</item>
        <mangName>도시대기</mangName>
        <year>1978</year>
        <addr>서울 강남구 학동로 426강남구청 별관 1동</addr>
        <stationName>강남구</stationName>
        <dmY>127.0472893</dmY>
      </item>
      <item>
        <dmX>37.504547</dmX>
        <item>SO2, CO, O3, NO2, PM10, PM2.5</item>
        <mangName>도시대기</mangName>
        <year>1982</year>
        <addr>서울 서초구 신반포로15길 16반포 2동 주민센터</addr>
        <stationName>서초구</stationName>
        <dmY>126.994611</dmY>
      </item>
      <item>
        <dmX>37.516083</dmX>
        <item>SO2, CO, O3, NO2, PM10, PM2.5</item>
        <mangName>도로변대기</mangName>
        <year>1995</year>
        <addr>서울 강남구 도산대로 104신사역2번출구 앞</addr>
        <stationName>도산대로</stationName>
        <dmY>127.019694</d

In [3]:
root = ET.ElementTree.fromstring(response_body)

df= pd.DataFrame()
for item in root.iter('item'):
    item_dict = {}
    item_dict['stationName'] = item.find('stationName').text
    item_dict['addr'] = item.find('addr').text
    item_dict['dmY'] = item.find('dmY').text
    df = df.append(item_dict, ignore_index=True)
    
df

AttributeError: 'NoneType' object has no attribute 'text'

In [4]:
cc = xmltodict.parse(response_body) # return collections.OrderedDict
dd = json.loads(json.dumps(cc)) # return dict
items = dd['response']['body']['items']['item']

df1 = pd.DataFrame(data=items)
df1


,dmX,item,mangName,year,addr,stationName,dmY
0,37.5175623,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1978,서울 강남구 학동로 426강남구청 별관 1동,강남구,127.0472893
1,37.504547,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1982,서울 서초구 신반포로15길 16반포 2동 주민센터,서초구,126.994611
2,37.516083,"SO2, CO, O3, NO2, PM10, PM2.5",도로변대기,1995,서울 강남구 도산대로 104신사역2번출구 앞,도산대로,127.019694
3,37.482867,"SO2, CO, O3, NO2, PM10, PM2.5",도로변대기,2009,서울특별시 서초구 강남대로 201서초구민회관 앞 중앙차로 (양재동),강남대로,127.035621
4,37.502685,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1984,서울 송파구 백제고분로 236삼전동 주민센터 (삼전동),송파구,127.092385
5,37.545089,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1997,서울 강동구 구천면로 42길 59천호1동 주민센터,강동구,127.136806
6,37.534035,"SO2, CO, O3, NO2, PM10, PM2.5",도로변대기,2008,서울 강동구 천호대로 1151길동사거리 강동성모요양병원 앞),천호대로,127.139172
7,37.452386,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1997,서울 금천구 금하로21길 20시흥5동 주민센터,금천구,126.908333
8,37.474899,"SO2, CO, O3, NO2, PM10, PM2.5",도로변대기,2018,서울 금천구 독산동 996-9한양수자인아파트 앞,시흥대로,126.898657
9,37.64793,"SO2, CO, O3, NO2, PM10, PM2.5",도시대기,1997,서울 강북구 삼양로 139길 49우이동 주민센터,강북구,127.011952


# 환경오염공단 대기오염정보
* 측정소별 측정정보 조회


In [5]:

enc = 'KJRTsEOIytUcYU%2BqXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx%2BrBGrAPjuxqrqHEJPbqdKeAlp8kNA%3D%3D'
dec = 'KJRTsEOIytUcYU+qXGApAmUl36YomGUuj2gWqy1ovmAE6E3Vz2ARuqccx+rBGrAPjuxqrqHEJPbqdKeAlp8kNA=='

url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMinuDustFrcstDspth'

params = '?' + urlencode({  quote_plus('serviceKey') : dec, 
                            quote_plus('returnType') : 'xml', 
                            quote_plus('searchDate') : '2021-09-23', 
                            quote_plus('InformCode') : ''})



request1 = Request(url + params)
request1.get_method = lambda: 'GET'
response_body1 = urlopen(request1).read().decode('utf-8') #DECODE 옵션넣어줘야 보임  
print(response_body1)

<?xml version="1.0" encoding="UTF-8"?>
<response>
  <header>
    <resultCode>00</resultCode>
    <resultMsg>NORMAL_CODE</resultMsg>
  </header>
  <body>
    <items>
      <item>
        <imageUrl4>https://www.airkorea.or.kr/file/proxyImage?fileName=2021/09/23/AQFv4_03h.20210923.KNU_09_01.PM2P5.1hsp.2021092303.png</imageUrl4>
        <informCode>PM10</informCode>
        <imageUrl5>https://www.airkorea.or.kr/file/proxyImage?fileName=2021/09/23/AQFv4_03h.20210923.KNU_09_01.PM2P5.1hsp.2021092309.png</imageUrl5>
        <imageUrl6>https://www.airkorea.or.kr/file/proxyImage?fileName=2021/09/23/AQFv4_03h.20210923.KNU_09_01.PM2P5.1hsp.2021092315.png</imageUrl6>
        <actionKnack/>
        <informCause>○ [미세먼지] 원활한 대기 확산으로 대기 상태가 대체로 청정할 것으로 예상됩니다.</informCause>
        <informOverall>○ [미세먼지] 전 권역이 &apos;좋음&apos;∼&apos;보통&apos;으로 예상됩니다.</informOverall>
        <informData>2021-09-23</informData>
        <informGrade>서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋음,경북 : 좋음,울산 : 좋음,대구 : 좋음,부산

In [7]:
ee = xmltodict.parse(response_body1) # return collections.OrderedDict
ff = json.loads(json.dumps(ee)) # return dict
items = ee['response']['body']['items']['item']

df2 = pd.DataFrame(data=items)
df2

,imageUrl4,informCode,imageUrl5,imageUrl6,actionKnack,informCause,informOverall,informData,informGrade,dataTime,imageUrl3,imageUrl2,imageUrl1
0,https://www.airkorea.or.kr/file/proxyImage?fil...,PM10,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 대체로 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'∼'보통'으로 예상됩니다.,2021-09-23,"서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 11시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
1,None,PM10,None,None,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'으로 예상됩니다.,2021-09-24,"서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 11시 발표,None,None,None
2,https://www.airkorea.or.kr/file/proxyImage?fil...,PM25,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 대체로 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'∼'보통'으로 예상됩니다.,2021-09-23,"서울 : 좋음,제주 : 보통,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 11시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
3,None,PM25,None,None,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'으로 예상됩니다.,2021-09-24,"서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 11시 발표,None,None,None
4,https://www.airkorea.or.kr/file/proxyImage?fil...,O3,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [오존] 대기오염물질의 광화학 반응에 의한 오존 생성과 이동으로 일부 충청지역과...,"○ [오존] 충남·전남·경남·제주권은 '나쁨', 그 밖의 권역은 '보통'으로 예상됩니다.",2021-09-23,"서울 : 보통,제주 : 나쁨,전남 : 나쁨,전북 : 보통,광주 : 보통,경남 : 나...",2021-09-23 11시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
5,https://www.airkorea.or.kr/file/proxyImage?fil...,PM10,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 대체로 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'∼'보통'으로 예상됩니다.,2021-09-23,"서울 : 좋음,제주 : 보통,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 05시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
6,None,PM10,None,None,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'으로 예상됩니다.,2021-09-24,"서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 05시 발표,None,None,None
7,https://www.airkorea.or.kr/file/proxyImage?fil...,PM25,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 대체로 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'∼'보통'으로 예상됩니다.,2021-09-23,"서울 : 좋음,제주 : 보통,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 05시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
8,None,PM25,None,None,None,○ [미세먼지] 원활한 대기 확산으로 대기 상태가 청정할 것으로 예상됩니다.,○ [미세먼지] 전 권역이 '좋음'으로 예상됩니다.,2021-09-24,"서울 : 좋음,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2021-09-23 05시 발표,None,None,None
9,https://www.airkorea.or.kr/file/proxyImage?fil...,O3,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,None,○ [오존] 대기오염물질의 광화학 반응에 의한 오존 생성과 이동으로 일부 충청지역과...,"○ [오존] 충남·전남·경남·제주권은 '나쁨', 그 밖의 권역은 '보통'으로 예상됩니다.",2021-09-23,"서울 : 보통,제주 : 나쁨,전남 : 나쁨,전북 : 보통,광주 : 보통,경남 : 나...",2021-09-23 05시 발표,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...,https://www.airkorea.or.kr/file/proxyImage?fil...
